In [1]:
from IPython.display import display, Image
import requests
import json
import pprint

from PIL import Image as Img
import numpy as np

In [5]:
%%time

files = {
         'country': 'rus',
        }

headers = {'Host': 'pyml.taxi.dev.yandex.net'}

# download and provide link to image of driver license here
img_file = './rus1.jpg'
with open(img_file, 'rb') as f:
    files['front_binary'] = f.read()

# url = 'http://127.0.0.1:5000/parse_driver_license'
# url = 'http://pyml.taxi.dev.yandex.net/parse_driver_license'
# url = 'http://pyml.taxi.tst.yandex.net/parse_driver_license'
url = 'http://pyml.taxi.yandex.net/parse_driver_license'
# url = 'http://target076i.load.yandex.net/parse_driver_license'

request = requests.post(url, files=files, headers=headers)
print(request, request.text)

recognized = json.loads(request.text)
display(recognized)

display(Image(img_file))

<Response [400]> Value for key == "front_binary" was not supplied in request


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [85]:
!curl {url} -F"front_binary=@{img_file}" -F"country=rus" -H "Host: pyml.taxi.dev.yandex.net"

{"due_date": {"confidence": 0.7114081979, "value": "2025-01-21"}, "first_name": {"confidence": 0.010445749274600746, "value": "\u041a\u043e\u043d\u0441\u0442\u0430\u043d\u0442\u0438\u043d"}, "last_name": {"confidence": 9.138888254243871e-07, "value": "\u0427\u0438\u0447\u0443\u043d\u043e\u0432"}, "middle_name": {"confidence": 0.03066553953711531, "value": "\u0410\u043d\u0430\u0442\u043e\u043b\u044c\u0435\u0432\u0438\u0447"}, "series": {"confidence": 0.820898056, "value": "5021"}, "number": {"confidence": 0.820898056, "value": "201811"}, "issue_date": {"confidence": 0.8247492313, "value": "2015-01-21"}, "country": "rus", "birth_date": {"confidence": 0.8185016513, "value": "1996-09-23"}}

In [34]:
pprint.pprint({"due_date": {"confidence": 0.7818428278, "value": "2021-10-11"}, "first_name": {"confidence": 0.01585803584630606, "value": "\u0420\u043e\u043c\u0430\u043d"}, "last_name": {"confidence": 7.938941600153221e-06, "value": "\u0421\u0430\u0442\u0430\u043b\u043a\u0438\u043d"}, "middle_name": {"confidence": 0.07352551526211905, "value": "\u0412\u043b\u0430\u0434\u0438\u043c\u0438\u0440\u043e\u0432\u0438\u0447"}, "series": {"confidence": 0.8297998905, "value": "2202"}, "number": {"confidence": 0.8297998905, "value": "813919"}, "issue_date": {"confidence": 0.7065529823, "value": "2011-10-11"}, "country": "rus", "birth_date": {"confidence": 0.8337495923, "value": "1986-09-19"}})

{'birth_date': {'confidence': 0.8337495923, 'value': '1986-09-19'},
 'country': 'rus',
 'due_date': {'confidence': 0.7818428278, 'value': '2021-10-11'},
 'first_name': {'confidence': 0.01585803584630606,
                'value': '\\u0420\\u043e\\u043c\\u0430\\u043d'},
 'issue_date': {'confidence': 0.7065529823, 'value': '2011-10-11'},
 'last_name': {'confidence': 7.938941600153221e-06,
               'value': '\\u0421\\u0430\\u0442\\u0430\\u043b\\u043a\\u0438\\u043d'},
 'middle_name': {'confidence': 0.07352551526211905,
                 'value': '\\u0412\\u043b\\u0430\\u0434\\u0438\\u043c\\u0438\\u0440\\u043e\\u0432\\u0438\\u0447'},
 'number': {'confidence': 0.8297998905, 'value': '813919'},
 'series': {'confidence': 0.8297998905, 'value': '2202'}}


# preparing ammo

In [1]:
from PIL import Image
import io
import werkzeug

In [2]:
img_str_dict = {}

with open('./rus1.jpg', 'rb') as f:
    img_str  = f.read()
    img = Image.open(io.BytesIO(img_str))
    img_str_dict['front'] = img_str

img_str_dict['country'] = 'rus'

In [3]:
ammo_str = \
""" 
POST /parse_driver_license HTTP/1.1
Host: pyml.taxi.yandex.net
Accept: */*
User-Agent: Tank
Content-Type: multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Length:

------WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Disposition: form-data; name="front_binary"; filename="front.jpg"
Content-Type: image/jpeg

{front}
------WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Disposition: form-data; name="country"; filename="country.txt"
Content-Type: application/json

{country}
------WebKitFormBoundary7MA4YWxkTrZu0gW--

""".format(**img_str_dict)

In [4]:
index = ammo_str.find('Content-Length:') + len('Content-Length:')
content_length = len(ammo_str[index:]) - 2
ammo_str = ammo_str[:index] + b' ' + str(content_length) + ammo_str[index:]
total_length = len(b'' + ammo_str.lstrip('\n')) - 2
ammo_str = b'' + str(total_length) + '' + ammo_str

In [5]:
with open('ammo_trial.txt', 'wb') as f:
    f.write(ammo_str.lstrip('\n'))